In [ ]:
import cv2
import glob
import os
import tqdm
import pybboxes as pbx
import numpy as np

#Save the image file
output_dark = "/home/ubuntum/Desktop/dark_helmet/"
output_light = "/home/ubuntum/Desktop/light_helmet/"

# Load the image file
image_folder = "/home/ubuntum/Desktop/helmet/helmet/"
image_files = sorted(glob.glob(os.path.join(image_folder, '*.jpg')))

# Load the text file
#txt_files = sorted(glob.glob("/home/ubuntum/Desktop/helmet/helmet_labels/*.txt"))



write_to_light_folder = False 
for image_file in tqdm.tqdm(image_files[:20000]):
   
    #image file and corresponding txt file to image
    image_basename = os.path.basename(image_file)
    txt_basename = os.path.splitext(image_basename)[0] + ".txt"
    txt_file = os.path.join("/home/ubuntum/Desktop/helmet/helmet_labels/", txt_basename)
    #print(txt_file)
    #print(image_file)
    with open(txt_file, 'r') as file:
        #print(txt_file)
        #print(image_file)
        lines = file.readlines()
        bboxes = []
        for line in lines:
            line = line.strip().split(" ")

            try:
                if int(line[0]) == 18:

                    # Load the image 
                    image = cv2.imread(image_file)

                    image_height, image_width  = image.shape[:2]
                    bbox = [float(x) for x in line[1:]]
                    
                    if int(line[0]) == 18:
                        box = pbx.convert_bbox(bbox, from_type='yolo', to_type='voc', image_size=(image_width,image_height))
                        bboxes.append(box)
                        

                else:
                    continue
            except ValueError:
                    #skip the line if the first element is not a valid integer
                    continue
        image_classified = False
        for bbox in bboxes:
            
            # Calculate the height of the ROI
            roi_height = bbox[3] - bbox[1]

            # Calculate the top and bottom y coordinates for the middle-upper part of the ROI
            middle_upper_y1 = bbox[1]
            middle_upper_y2 = int(bbox[1] + roi_height * 1)

            # Set vertical offset for cropping
            offset = 1

            # Extract ROI from image
            roi = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            
            # Apply thresholding to ROI
            try:
                # Convert ROI to HSV color space
                hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                # Split into individual channels
                hue, saturation, values = cv2.split(hsv)
                # Apply adaptive thresholding to value channel
                thresh_value = cv2.adaptiveThreshold(hue, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 2)
                
                # Crop the right and left upper corners of the thresholded image
                #cropped_thresh_value = thresh_value[:, 1:-1]
                #cropped_thresh_value2 = cropped_thresh_value[offset:, :]
                # Merge thresholded value channel with hue and saturation
                segmented = thresh_value, saturation, values
                
                #Show threshold image
                #if cropped_thresh_value2.shape[0] > 0 and cropped_thresh_value2.shape[1] > 0:
                #    cv2.imshow("threshold", cropped_thresh_value2)
                #    cv2.waitKey(0)

                # Calculate the number of pixels in the thresholded image inside the bounding box
                num_pixels = np.sum(segmented[2] > 0)
            
                # Determine how many pixels are close to black and how many are close to white
                num_black_pixels = np.sum(segmented[2] < 60)

                num_white_pixels = num_pixels - num_black_pixels
                new_num_white_pixels = abs(num_white_pixels)
                white_percentage = (new_num_white_pixels * 100) / num_pixels 

                print("sum of  pixels:", image_file,  num_pixels)
                print("sum of black pixels:", num_black_pixels)
                print("sum of white pixels:", new_num_white_pixels)
                print("white percentage:", white_percentage)

                if white_percentage > 60:
                    
                    #base_name, ext = os.path.splitext(os.path.basename(image_file))
                    #out_light = os.path.join(output_light, f"{base_name}{ext}")          
                    #cv2.imwrite(out_light, image)
                    image_classified = True
                    break

            except cv2.error as e:
                print(f"cv2.cvtColor() error: {e}")
                print(image_file)
                continue

        
        # Check if image has already been classified
        if not image_classified:
            # Save image to out_dark folder               
            base_name, ext = os.path.splitext(os.path.basename(image_file))
            out_dark = os.path.join(output_dark, f"{base_name}{ext}")          
            cv2.imwrite(out_dark, image)
cv2.destroyAllWindows()
